In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# specify substep parameters for interactive run
# this cell will be replaced during job run with the parameters from json within params subfolder
substep_params={
}

In [3]:
# load pipeline and step parameters - do not edit
from sinara.substep import get_pipeline_params, get_step_params
pipeline_params = get_pipeline_params(pprint=True)
step_params = get_step_params(pprint=True)

**Pipeline params:**


{'env_name': 'user',
 'pipeline_name': 'example_median_house_price',
 'zone_name': 'test'}




**Step params:**


{}




In [4]:
#3 define substep interface
from sinara.substep import NotebookSubstep, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params)

substep.interface(
    outputs = 
    [
        { ENTITY_NAME: "boston_dataset" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

**STEP NAME:**


'data_load'




**OUTPUTS:**


[{'user.example_median_house_price.test.data_load.boston_dataset': '/data/home/jovyan/example_median_house_price/test/data_load/run-24-06-10-110046/boston_dataset'}]




In [5]:
#4 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

Session is run


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/10 11:00:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Dataset description:
The Boston house-price data of Harrison, D. and Rubinfeld, D.L. 'Hedonic prices and the demand for clean air', J. Environ. Economics & Management, vol.5, 81-102, 1978.   Used in Belsley, Kuh & Welsch, 'Regression diagnostics...', Wiley, 1980.   N.B. Various transformations are used in the table on pages 244-261 of the latter.<br>

## Feature columns in order:  
**TRACT**, a statistical division of the area that is used by researchers to break down towns and cities.<br>
**LON** and LAT are the longitude and latitude of the center of the census tract.<br>
**MEDV** is the median value of owner-occupied homes, in thousands of dollars.<br>
**CRIM** is the per capita crime rate<br>
**ZN** is related to how much of the land is zoned for large residential properties<br>
**INDUS** is proportion of area used for industry<br>
**CHAS** is 1 if the census tract is next to the Charles River<br>
**NOX** is the concentration of nitrous oxides in the air<br>
**RM** is the average number of rooms per dwelling<br>
**AGE** is the proportion of owner-occupied units built before 1940<br>
**DIS** is a measure of how far the tract is from centers of employment in Boston<br>
**RAD** is a measure of closeness to important highways<br>
**TAX** is the property tax rate per $10,000 of value<br>
**PTRATIO** is the pupil-teacher ratio by town<br>

## Dataset load process
1. Download dataset from the internet storage
1. Fix columns data and orderlumns data and order

In [6]:
#5 load csv dataset into pandas dataframe

import pandas as pd

csv_url = "https://ocw.mit.edu/courses/15-071-the-analytics-edge-spring-2017/d4332a3056f44e1a1dec9600a31f21c8_boston.csv"
csv_feature_columns = ['TRACT', 'LON', 'LAT', 'MEDV', 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO']
boston_df = pd.read_csv(csv_url, sep=",")[csv_feature_columns]

In [7]:
#7 convert pandas dataframe to spark dataframe

boston_df_spark = spark.createDataFrame(boston_df)

In [8]:
#8 write outputs
outputs = substep.outputs()

boston_df_spark.write.parquet(outputs.boston_dataset, mode='overwrite')

In [9]:
#9 stop spark
SinaraSpark.stop_session()